1: Convert pdf to an image. Store the page as an image  

2: img processing techniques like grayscale and diagram removal  

3: bounding boxes per word  

4: combine boxes into lines of text  


In [12]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
from PIL import Image
import PyPDF2
import pytesseract
from pdf2image import convert_from_path


# Step 1: Insert pdf path and save as image  
Must install poppler: https://github.com/oschwartz10612/poppler-windows/releases  
Follow directions to add poppler:  
- Download poppler as .zip, then extract all
- Press Win + R, type sysdm.cpl, hit Enter.
- Go to Advanced → Environment Variables.
- Under User variables, find Path → Edit → New.
- Paste: C:\pasteYourPathHere\poppler-23.10.0\bin

Restart terminal, verify instalation: pdftoppm -h

In [13]:
pdf_path = r"C:\Users\dkhun\UC Davis\AISC Github repository\BeginnerProjectFallQuarter2025\data\raw_pdfs\textbook_pdf_6.pdf"
pdf_image = convert_from_path(pdf_path)

pages = list(pdf_image)
print(len(pages))
#pages[0].show()

#resize image to standard?


2


# Step 2: Image processing shi

In [ ]:
#convert image to NumPY array in order to apply filters
opencv_page1 = np.array(pages[0])
opencv_page1 = cv2.cvtColor(opencv_page1, cv2.COLOR_RGB2BGR)

#Apply grayscale, bilateral (to retain edges better), adaptive binary filter
gray_page1 = cv2.cvtColor(opencv_page1, cv2.COLOR_BGR2GRAY)
#If overblurred, decrease sigmaColor and sigmaSpace (to 50?)
bilateral_page1 = cv2.bilateralFilter(gray_page1, d=9, sigmaColor=75, sigmaSpace=75)
binary_filter = cv2.adaptiveThreshold(
    bilateral_page1, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 7, 5)
#gaussian_blur_page1 = cv2.GaussianBlur(gray_page1,(3,3),0)
kernel = np.ones((1, 1), np.uint8)
morphological_filter = cv2.morphologyEx(binary_filter, cv2.MORPH_CLOSE, kernel)

#Improve edge sharpness by subtracting low freq parts from original image
#This leaves high frequency (edges)
#Then, add high freq back to original to get sharper edges
high_freq_page1 = cv2.subtract(gray_page1, bilateral_page1) #chat reblurred the image, I used gaussian_blur_page1
sharpen_strength = 15.0
sharpened_page1 = cv2.addWeighted(gray_page1, 1.0, high_freq_page1, sharpen_strength, 0)


#Convert the opencv file back to PIL so that it can be viewed
test = Image.fromarray(sharpened_page1)
test1 = Image.fromarray(morphological_filter)
test.show()
test1.show()


# Monday 10/20 Meeting  
Want bounding boxes merged into lines. Each line is a specific png image  
- Use pandas dataframe to represent data (makes it more efficient)  
- Use pd.readcsv() ??  
- csv (excel) file has data for things like left_bounding, width, height, confidence, and text PER WORD BOX  
- Important: print(df.colums) says names of colums  
- Important: print(df.head(n)) gives all the data for n rows  
To access all the values in certain columns (ex: left bounding), use df_left = df['left_bounding'][][]  
- Or use df_column_names = df[['col1,'col2','coln']]  
For loops:  
- "for idx,row in df.iterrows() is used to iterate through the rows  
- Initialize right pixel as negative infinity  
- if idx=0: right_pixel = row['left'] + row['width']
     - right pixel = left bounding + width  
     - create threshold for if two words are close enough (sequential) and test vetical distance(same line)  
- Make else contained within the for loop: save segment of pdf (segment=boxed line of text), specify them by combined values

# To Do:   
1) Filtering: get text clear and remove images/diagrams  
- sharp edges for text  
2) Find and implement a library for bounding boxes  
3) write bandas logic to create single-line boxes (merging of individual boxes)